# Riemann Hypothesis Numerical Validation

🧠 **Copilot Prompt:**

Este cuaderno valida numéricamente la fórmula explícita propuesta en el artículo principal.
Por favor, sugiere:

- Comparación dinámica entre las versiones de A_infty y zero_sum.
- Gráficos de convergencia de cada suma.
- Tabla resumen con errores relativos y absolutos.

Ejecuta automáticamente con nbconvert y exporta a docs/validation.html.

## Dynamic Comparison Framework

This notebook provides dynamic validation of different versions of the Riemann zeta function explicit formula validation, with interactive convergence analysis and comprehensive error reporting.

In [ ]:
import mpmath as mp
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from IPython.display import display, HTML
import warnings

# Set high precision
mp.mp.dps = 50

print(f"🔧 mpmath precision: {mp.mp.dps} decimal places")
print(f"📊 Notebook ready for validation")

## Test Functions Definition

We define three different test functions to validate the explicit formula:

In [ ]:
# Test function definitions
def f1(u, a=3.0, sigma=1.0): 
    """Truncated Gaussian f1"""
    return mp.exp(-u**2/2) if abs(u) <= a else mp.mpf(0)

def f2(u, a=2.0): 
    """Truncated Gaussian f2 with steeper decay"""
    return mp.exp(-u**2) if abs(u) <= a else mp.mpf(0)

def f3(u, a=2.0): 
    """Polynomial test function f3"""
    return (1 - u**2/4)**2 if abs(u) <= a else mp.mpf(0)

# Mellin transform helper
def fhat(f, s, lim):
    """Numerical Mellin transform"""
    return mp.quad(lambda u: f(u) * mp.exp(s * u), [-lim, lim], maxdegree=10)

print("✅ Test functions defined")

## Core Validation Functions

Implementation of the different sides of the explicit formula:

In [ ]:
def prime_sum_v1(f, P=1000, K=50):
    """Prime side calculation - Version 1 (Original)"""
    s = mp.mpf(0)
    import sympy as sp
    primes = list(sp.primerange(2, 7919))[:P]
    for p in primes:
        lp = mp.log(p)
        for k in range(1, K+1):
            s += lp * f(k * lp)
    return s

def prime_sum_v2(f, P=1000, K=50):
    """Prime side calculation - Version 2 (Optimized)"""
    s = mp.mpf(0)
    # Use mpmath's prime generation for consistency
    for i in range(1, P+1):
        try:
            p = mp.prime(i)
            lp = mp.log(p)
            for k in range(1, K+1):
                s += lp * f(k * lp)
        except:
            break
    return s

def A_infty_v1(f, sigma0=2.0, lim=3, T=50):
    """Archimedean contribution - Version 1 (Original)"""
    def integrand(t):
        s = mp.mpc(sigma0, t)
        return (mp.digamma(s/2) - mp.log(mp.pi)) * fhat(f, s, lim)
    integ = mp.quad(integrand, [-T, T], maxdegree=10) / (2 * mp.pi)
    res1 = fhat(f, mp.mpf(1), lim) / mp.mpf(1)
    return integ - res1

def A_infty_v2(f, sigma0=2.0, lim=3, T=100):
    """Archimedean contribution - Version 2 (Higher precision)"""
    def integrand(t):
        s = mp.mpc(sigma0, t)
        kernel = mp.digamma(s/2) - mp.log(mp.pi)
        transform = fhat(f, s, lim)
        return kernel * transform
    
    # Higher precision integration
    with mp.workdps(mp.mp.dps + 10):
        integ = mp.quad(integrand, [-T, T], maxdegree=20, error=True)
        res1 = fhat(f, mp.mpf(1), lim) / mp.mpf(1)
        return (integ[0] / (2j * mp.pi) - res1).real

def zero_sum_v1(f, N=100, lim=3):
    """Zero sum - Version 1 (Basic)"""
    s = mp.mpf(0)
    for n in range(1, N+1):
        rho = mp.zetazero(n)
        s += fhat(f, mp.im(rho) * 1j, lim).real
    return s

def zero_sum_v2(f, N=100, lim=3):
    """Zero sum - Version 2 (Optimized with progress tracking)"""
    s = mp.mpf(0)
    zeros_used = []
    
    for n in range(1, N+1):
        rho = mp.zetazero(n)
        gamma = mp.im(rho)
        zeros_used.append(float(gamma))
        
        # Use purely imaginary argument for the transform
        contrib = fhat(f, 1j * gamma, lim).real
        s += contrib
        
    return s, zeros_used

print("✅ Validation functions defined (v1 and v2 versions)")

## Dynamic Comparison Analysis

In [ ]:
def run_comparative_validation(functions_dict, N_zeros_list=[50, 100, 200], versions=['v1', 'v2']):
    """Run comprehensive validation comparing different versions and parameters"""
    
    results = []
    
    for func_name, (func, lim) in functions_dict.items():
        print(f"\n🧪 Testing function: {func_name}")
        
        for N in N_zeros_list:
            print(f"  📊 N_zeros = {N}")
            
            for version in versions:
                try:
                    print(f"    🔄 Version {version}...")
                    
                    # Select version functions
                    if version == 'v1':
                        prime_fn = prime_sum_v1
                        arch_fn = A_infty_v1
                        zero_fn = zero_sum_v1
                    else:
                        prime_fn = prime_sum_v2
                        arch_fn = A_infty_v2
                        zero_fn = zero_sum_v2
                    
                    # Calculate components
                    prime_sum = prime_fn(func, P=min(500, N), K=20)
                    arch_sum = arch_fn(func, lim=lim, T=min(50, N//2))
                    
                    if version == 'v2':
                        zero_sum, zeros_list = zero_fn(func, N=N, lim=lim)
                    else:
                        zero_sum = zero_fn(func, N=N, lim=lim)
                        zeros_list = None
                    
                    # Total arithmetic side
                    arithmetic_total = prime_sum + arch_sum
                    
                    # Calculate errors
                    error_abs = abs(arithmetic_total - zero_sum)
                    error_rel = error_abs / abs(arithmetic_total) if arithmetic_total != 0 else mp.mpf('inf')
                    
                    result = {
                        'function': func_name,
                        'version': version,
                        'N_zeros': N,
                        'prime_sum': float(prime_sum),
                        'arch_sum': float(arch_sum), 
                        'arithmetic_total': float(arithmetic_total),
                        'zero_sum': float(zero_sum),
                        'error_abs': float(error_abs),
                        'error_rel': float(error_rel),
                        'zeros_used': zeros_list
                    }
                    
                    results.append(result)
                    
                    print(f"      ✅ Error: {error_abs:.2e} (rel: {error_rel:.2e})")
                    
                except Exception as e:
                    print(f"      ❌ Error in {version}: {e}")
                    continue
    
    return pd.DataFrame(results)

# Define test functions
test_functions = {
    'f1': (f1, 3.0),
    'f2': (f2, 2.0),
    'f3': (f3, 2.0)
}

print("✅ Comparative validation framework ready")

## Run Validation and Generate Results

In [ ]:
# Run the comparative validation
print("🚀 Starting comprehensive validation...")

validation_results = run_comparative_validation(
    test_functions, 
    N_zeros_list=[25, 50, 100],  # Reduced for demo
    versions=['v1', 'v2']
)

print("\n📊 Validation completed!")
print(f"Total results: {len(validation_results)}")

## Results Summary Table

In [ ]:
# Create summary table
summary_df = validation_results.pivot_table(
    index=['function', 'N_zeros'],
    columns='version', 
    values=['error_abs', 'error_rel'],
    aggfunc='mean'
).round(8)

print("📋 Error Summary by Function and Version:")
display(HTML(summary_df.to_html(classes='table table-striped')))

# Detailed results table
print("\n📈 Detailed Results:")
display_cols = ['function', 'version', 'N_zeros', 'arithmetic_total', 'zero_sum', 'error_abs', 'error_rel']
display(HTML(validation_results[display_cols].to_html(classes='table table-striped', index=False)))

## Convergence Analysis Plots

In [ ]:
# Plot convergence analysis
plt.figure(figsize=(15, 10))

# Error vs N_zeros plot
plt.subplot(2, 2, 1)
for func in validation_results['function'].unique():
    for version in validation_results['version'].unique():
        data = validation_results[
            (validation_results['function'] == func) & 
            (validation_results['version'] == version)
        ]
        plt.semilogy(data['N_zeros'], data['error_abs'], 
                    marker='o', label=f'{func}_{version}', alpha=0.7)

plt.xlabel('Number of Zeros')
plt.ylabel('Absolute Error')
plt.title('Convergence: Error vs Number of Zeros')
plt.legend()
plt.grid(True, alpha=0.3)

# Version comparison
plt.subplot(2, 2, 2)
for func in validation_results['function'].unique():
    v1_data = validation_results[
        (validation_results['function'] == func) & 
        (validation_results['version'] == 'v1')
    ]
    v2_data = validation_results[
        (validation_results['function'] == func) & 
        (validation_results['version'] == 'v2')
    ]
    
    if not v1_data.empty and not v2_data.empty:
        plt.semilogy(v1_data['N_zeros'], v1_data['error_abs'], 
                    'o--', label=f'{func}_v1', alpha=0.7)
        plt.semilogy(v2_data['N_zeros'], v2_data['error_abs'], 
                    's-', label=f'{func}_v2', alpha=0.7)

plt.xlabel('Number of Zeros')
plt.ylabel('Absolute Error')
plt.title('Version Comparison (v1 vs v2)')
plt.legend()
plt.grid(True, alpha=0.3)

# Relative error heatmap
plt.subplot(2, 2, 3)
pivot_rel = validation_results.pivot_table(
    index='function', columns='N_zeros', values='error_rel', aggfunc='mean'
)
import seaborn as sns
sns.heatmap(pivot_rel, annot=True, fmt='.2e', cmap='YlOrRd')
plt.title('Relative Error Heatmap')

# Component contribution
plt.subplot(2, 2, 4)
sample_result = validation_results.iloc[0]
components = ['prime_sum', 'arch_sum', 'zero_sum']
values = [abs(sample_result[comp]) for comp in components]
plt.bar(components, values, alpha=0.7)
plt.yscale('log')
plt.title(f'Component Magnitudes ({sample_result["function"]})')
plt.xticks(rotation=45)

plt.tight_layout()
plt.savefig('docs/validation_plots.png', dpi=300, bbox_inches='tight')
plt.show()

print("📊 Convergence plots generated")

## Export Results

In [ ]:
# Save detailed results
import os
os.makedirs('data', exist_ok=True)
os.makedirs('docs', exist_ok=True)

# Export CSV
validation_results.to_csv('data/validation_results_detailed.csv', index=False)

# Export summary
summary_df.to_csv('data/validation_summary.csv')

# Create HTML report
html_report = f"""
<!DOCTYPE html>
<html>
<head>
    <title>Riemann Hypothesis Validation Report</title>
    <style>
        body {{ font-family: Arial, sans-serif; margin: 40px; }}
        table {{ border-collapse: collapse; width: 100%; }}
        th, td {{ border: 1px solid #ddd; padding: 8px; text-align: left; }}
        th {{ background-color: #f2f2f2; }}
        .pass {{ color: green; font-weight: bold; }}
        .fail {{ color: red; font-weight: bold; }}
    </style>
</head>
<body>
    <h1>🧮 Riemann Hypothesis Numerical Validation Report</h1>
    <p><strong>Generated:</strong> {pd.Timestamp.now()}</p>
    <p><strong>Total tests:</strong> {len(validation_results)}</p>
    
    <h2>📊 Summary Results</h2>
    {summary_df.to_html(classes='table table-striped')}
    
    <h2>📈 Detailed Results</h2>
    {validation_results[display_cols].to_html(classes='table table-striped', index=False)}
    
    <h2>🎯 Test Status</h2>
    <ul>
"""

# Add test status
for _, row in validation_results.iterrows():
    status = "✅ PASS" if row['error_abs'] < 1e-3 else "❌ FAIL"
    html_report += f"<li>{row['function']} {row['version']} (N={row['N_zeros']}): {status}</li>\n"

html_report += """
    </ul>
</body>
</html>
"""

with open('docs/validation.html', 'w') as f:
    f.write(html_report)

print("✅ Results exported:")
print("  - data/validation_results_detailed.csv")
print("  - data/validation_summary.csv")
print("  - docs/validation.html")
print("  - docs/validation_plots.png")